In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

# **Data preprocessing**

In [ ]:
train=pd.read_csv('/gdrive/My Drive/Colab Notebooks/competition_data (1)/train.csv') 
test=pd.read_csv('/gdrive/My Drive/Colab Notebooks/competition_data (1)/test.csv')
submission=pd.read_csv('/gdrive/My Drive/Colab Notebooks/competition_data (1)/sample_submission.csv', index_col=0)

#결측치 확인
train.info()

In [ ]:
#nan값 전부 0으로 삽입
train.fillna(0, inplace= True)
test.fillna(0, inplace= True)

In [ ]:
#상관관계가 적은 시간 특성, 인덱스 col을 제거

drop_list = ['introelapse', 'testelapse', 'surveyelapse', 'index',]

train.drop(drop_list, axis = 1, inplace = True)
test.drop(drop_list, axis = 1, inplace = True)

In [ ]:
#0값을 해당 특성의 train의 평균값으로 처리
processing_feature = ['gender', 'married', 'education', 'voted', 'urban', 'orientation', 'religion', 'engnat', 'ASD']

for pro in processing_feature:
    train.loc[train[pro] == 0, pro] = train[pro].mean()
    test.loc[test[pro] == 0, pro] = train[pro].mean()

In [ ]:
#age 이상치(80살 이상) 0으로 처리
train.loc[train['age'] > 80, 'age'] = 0
test.loc[test['age'] > 80, 'age'] = 0

In [ ]:
#age의 이상치는 train데이터의 평균값으로 처리
train.loc[train['age'] == 0, 'age'] = train['age'].mean()
test.loc[test['age'] == 0, 'age'] = train['age'].mean()

#familysize 6 이상인 값 최빈값으로 교체, 
train.loc[train['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]
test.loc[test['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]

In [ ]:
#훈련 데이터에 존재하는 상위 6개의 나라
best6_country = train['country'].value_counts().keys()[:6]

#train데이터 상위 6개 국가 제외 train데이터의 최빈값으로 처리
for i in range(len(train['country'])):
    if train['country'][i] not in best6_country or train['country'][i]==0:
        train['country'][i] = train['country'].mode()[0]

#test데이터 상위 6개 국가 제외 train데이터의 최빈값으로 처리
for i in range(len(test['country'])):
    if test['country'][i] not in best6_country or test['country'][i]==0:
        test['country'][i] = train['country'].mode()[0]

In [ ]:
#country 특성 원-핫인코딩 수행 (country 특성 세부 분할)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# **Hyperparameter Tuning: Optuna**

In [ ]:
!pip install --quiet optuna

In [ ]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner

In [ ]:
features = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'country_AUS',
       'country_CAN', 'country_DEU', 'country_GBR', 'country_PHL',
       'country_USA']

target = ['nerdiness']

In [ ]:
df_trains = []
df_valids = []

skf = StratifiedKFold(n_splits=10, random_state=2022, shuffle=True)
for train_index, valid_index in skf.split(train[features], train[target]):
    df_train = train.loc[train_index]
    df_valid = train.loc[valid_index]
    df_trains.append(df_train)
    df_valids.append(df_valid)

    x_train = df_train[features]
    y_train = df_train[target]

    x_test = df_valid[features]
    y_test = df_valid[target]

In [ ]:
def accuracy(true, pred):
    return np.mean(true==pred)

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 300, 824, step=1, log=True), 
        'max_depth': trial.suggest_int('max_depth', 10, 20, step=1, log=False), 
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True), 
        'n_estimators': trial.suggest_int('n_estimators', 1500, 3000, step=1, log=True), 
        "metric": "multi_auc",
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=1, log=False), 
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 1.0),
        'random_state': 2022
    }
    
    score = []
    for df_train, df_valid in zip(df_trains, df_valids):
        clf = LGBMClassifier(**params)
        clf.fit(x_train, y_train)
        
        pred = clf.predict_proba(df_valid[features])[:, 1]
        true = df_valid[target].values
        score.append(roc_auc_score(true, pred))
    score = np.mean(score)
    return score

# Hyperparameter Tuning
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=2022), pruner=SuccessiveHalvingPruner())
study.optimize(objective, n_trials=10)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
clfs = []
for df_train in df_trains:
    clf = LGBMClassifier(**study.best_params)
    clf.fit(df_train[features], df_train[target])
    pred_clf = clf.predict_proba(x_test)[:, 1]
    print(roc_auc_score(y_test, pred_clf))
    clfs.append(clf)

In [ ]:
clfs

In [ ]:
pred = [clf.predict_proba(test[features]) for clf in clfs]

In [ ]:
real_pred = []

for i in range(10):
    for j in range(len(pred[i])):
        real_pred.append(pred[i][j][1])

In [ ]:
real_pred = []

for i in range(1):
    for j in range(len(pred[i])):
        real_pred.append(pred[i][j][1])

for i in range(2,10):
    for j in range(len(pred[i])):
        real_pred[j] += pred[i][j][1]
        if i==9:
            real_pred[j]= real_pred[j]/10

real_pred[:10]

In [ ]:
x_train = train.drop('nerdiness', axis = 1)
y_train = train['nerdiness']

In [ ]:
#extree model

extraTree_model = ExtraTreesClassifier(n_jobs=-1, random_state = 2022, n_estimators=3000)
extraTree_model.fit(x_train, y_train)
extraTree_pred = extraTree_model.predict_proba(test)[:, 1]

In [ ]:
real2_pred = list(extraTree_pred)

# **Ensemble**

In [ ]:
result_pred = []

for i in range(len(real2_pred)):
    result_pred.append((real2_pred[i] * 0.6) + (real_pred[i] * 0.4))

result_pred

In [ ]:
submission['nerdiness'] = result_pred